In [8]:
%load_ext autoreload
%autoreload 2

import os, time, pickle
from scripts.tools.issuetracker import IssuetrackerAPI
from scripts.tools.issuetracker.issues import Issues
from jpnotebooks.Software.frs_tools import extract, matchers
from jpnotebooks.Software.frs_tools.extract import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def expired(loaded, expires):
    now = time.time()
    age = (now-loaded)/3600
    return age > expires

class IssueCache():
    def __init__(self, issues=None, dltime=None, expires=12, api=None):
        self.issues = None
        self._parse_issues(issues, api)
        self.loaded = dltime or time.time()
        self.expires = expires
        
    def _parse_issues(self, issues, api):
        if isinstance(issues, dict):
            self.issues = Issues(api, list(issues.values()))
        elif issues:
            self.issues = Issues(api, list(issues))
            
    def age(self):
        return self._age(self.loaded)
    
    def get(self):
        return self.issues.find()  # copies all issues
    
    @classmethod
    def from_dl(cls, api, *a, expires=12, **k):
        s = cls(expires=expires)
        s.reload(api, *a, **k)
        return s
    
    def reload(self, api, *a, **k):
        iss = api.download_issues(*a, **k)
        self._parse_issues(iss, api)
        self.loaded = time.time()
    
    def _age(self, t):
        now = time.time()
        hrs = (now - t)/3600
        return hrs        
    
    def expired(self):
        return self.age() > self.expires
    
    def save(self, fn):
        with open(fn, 'wb') as f:
            pickle.dump((self.loaded, self.issues), f)
            
    @classmethod
    def open(cls, fn, api, *a, expires=12, **k):
        try:
            s = cls.load(fn, expires)
        except (FileNotFoundError, EOFError, pickle.PickleError):
            s = cls(None, -1)  # trigger expired
            
        if s.expired():
            s.reload(api, *a, **k)
            s.save(fn)
        return s
        
    @classmethod
    def load(cls, fn, expires=12):
        with open(fn, 'rb') as f:
            ob = pickle.load(f)
        loaded, issues = ob
        return cls(issues, loaded, expires)     

In [15]:
it = IssuetrackerAPI("issue.pbsbiotech.com", "nstarkweather@pbsbiotech.com", "kookychemist")
sf = "./.issuetracker/issue_cache.pkl"

def refresh(sf=sf, it=it):
    global c, issues
    c = IssueCache.from_dl(it, "pbssoftware")
    c.save(sf)
    issues = Issues(it, c.get())
    return c.get()

if not os.path.exists(sf):
    os.makedirs(os.path.dirname(sf), exist_ok=True)
c = IssueCache.open(sf, it, "pbssoftware")
iss = c.get()
issues = Issues(it, iss)

In [11]:
refresh()

<Issues Object: 504 Issues>

In [16]:
import sys
class ReleaseNotesFormatter():
    def __init__(self, issues, header=True):
        self.issues = issues
        self.release_notes = []
        self.sections = []
        self.include_header = header
        self.parse()
        
    def check(self, **kw):
        if not self.sections:
            self._extract()
        
        def match(i):
            for k,v in kw.items():
                if isinstance(v, tuple):
                    if getattr(i, k) not in v:
                        return False
                else:
                    if getattr(i, k) != v:
                        return False
            return True
        
        iset = {i.id for i in self.issues if match(i)}
        iset2 = {t[0].id for t in self.sections}
        diff = iset - iset2
        if diff:
            print("Issues without Release Notes:")
            ldiff = sorted(diff)
            for i in ldiff:
                print(" ", i)
            return ldiff
        return []
        
    def _extract(self):
        extractor = extract.IssuesExtractor(self.issues)
        self.sections = extractor.extract("Release Notes")
        
    def parse(self):
        self._extract()
        self.release_notes = rnl = []
        for issue, notes in self.sections:
            if notes.lower() == "* n/a":
                continue
            if self.include_header:
                rns = "%s:\n%s"%(_header(issue), notes)
            else:
                rns = notes
            rnl.append(rns)
            
    def write(self, b):
        b.write("\n".join(self.release_notes))
        
    def write2(self, fn):
        with open(fn, 'w') as f:
            self.write(f)
        
    def print(self):
        self.write(sys.stdout)
        
    def parse2(self):
        self.parse()
        self.print()
        
    def categorize(self, *cat):
        # attempt to categorize issues by subject based on provided categories
        self.release_notes = rnl = []
        cat = [c.lower() for c in cat]
        cats = {c: [] for c in cat}
        cats["other"] = []
        cats["Bug Fixes"] = []
        for iss, notes in self.sections:
            key = None
            if iss.tracker == "Bug":
                key = "Bug Fixes"
            else:
                sub = iss.subject.lower()
                for c in cat:
                    if c in sub:
                        key = c
                        break
                else:
                    key = "other"
            if notes.lower() == "* n/a":
                continue
            lines = notes.splitlines()
            #lines2 = ["  %d - %s"%(iss.id, iss.subject)]
            lines2 = []
            for l in lines:
                stars, note = l.split(" ", 1)
                note = "    "*(len(stars)) + "* " + note
                lines2.append(note)
            #lines2.append("\n")
            cats[key].append(lines2)
        
        for category, notes in cats.items():
            rnl.append("\n%s:"%category.title())
            for inotes in notes:
                for line in inotes:
                    rnl.append(line)
                    
    def categorize2(self, *cat):
        # attempt to categorize issues by subject based on provided categories
        self.release_notes = rnl = []
        for iss, notes in self.sections:
            key = None
            if notes.lower() == "* n/a":
                continue
            if iss.tracker == "Bug":
                key = "bug fixes"
            else:
                sub = iss.subject.lower()
                for c in cat:
                    c = c.lower()
                    if c in sub:
                        key = c
                        break
                else:
                    key = "other"
            lines = notes.splitlines()
            for l in lines:
                stars, note = l.split(" ", 1)
                note = "|".join((key, str(iss.id), stars, note))
                rnl.append(note)
                    
def _header(issue):
    return "Issue #%d - %s - %s"%(issue.id, issue.tracker, issue.subject)

In [20]:
categories = "Agitation", "Pumps", "Temperature", "DO", "pH", "MFCs", "Shell", "Desktop", "Hello"
sprint = "3.0"
formatter = ReleaseNotesFormatter(issues.find2(sprint_milestone=sprint), False)
formatter.categorize2(*categories)
formatter.write2("release notes test %s categorize.txt"%sprint)

Duplicate Key Warning ('FRS1234')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Motivation')
Got 5 warning(s) while parsing issue #3287



In [18]:
formatter = ReleaseNotesFormatter(issues.find2(sprint_milestone=sprint), False)
unchecked = formatter.check()

for m in unchecked:
    url = "https://issue.pbsbiotech.com/issues/%s"%str(m)
    print("Issue %s: %s"%(m,url))

Duplicate Key Warning ('FRS1234')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Motivation')
Got 5 warning(s) while parsing issue #3287



In [18]:
# import time
# import webbrowser
# for i, m in enumerate(unchecked, 1):
#     url = "https://issue.pbsbiotech.com/issues/%s"%str(m)
#     if not i%5: input("Press enter to continue")
#     time.sleep(0.5)
#     webbrowser.open_new_tab(url)